In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv('tweets.csv', lineterminator='\n')

In [3]:
df

,Date,Hashtag,Tweet
0,2022-11-03 15:03:47+00:00,#Apple,USB-C on an iPhone is probably not gonna happe...
1,2022-11-03 15:03:27+00:00,#Apple,"Guey, que pedo con #Apple no se si sea el únic..."
2,2022-11-03 15:03:01+00:00,#Apple,#Apple (@Apple) Music offre un petit cadeau au...
3,2022-11-03 15:02:58+00:00,#Apple,I am using #老虎证券. Check out #Apple (AAPL). Pri...
4,2022-11-03 15:02:44+00:00,#Apple,Accès coupé à OQEE sur #Apple (@Apple) TV à ca...
...,...,...,...
499995,2022-03-23 04:11:42+00:00,#billgates,#moderna #fauci #BillGates #Soros #epstein par...
499996,2022-03-23 02:38:44+00:00,#billgates,@BMi_99999 @GillesWell #BillGates subventionne...
499997,2022-03-23 02:16:04+00:00,#billgates,"#ATENCION 🌐| El globalista, biólogo, virólogo,..."
499998,2022-03-23 01:39:34+00:00,#billgates,There is going to be #exposure of #BillGates. ...


In [4]:
X = df['Tweet']

In [5]:
X

0         USB-C on an iPhone is probably not gonna happe...
1         Guey, que pedo con #Apple no se si sea el únic...
2         #Apple (@Apple) Music offre un petit cadeau au...
3         I am using #老虎证券. Check out #Apple (AAPL). Pri...
4         Accès coupé à OQEE sur #Apple (@Apple) TV à ca...
                                ...                        
499995    #moderna #fauci #BillGates #Soros #epstein par...
499996    @BMi_99999 @GillesWell #BillGates subventionne...
499997    #ATENCION 🌐| El globalista, biólogo, virólogo,...
499998    There is going to be #exposure of #BillGates. ...
499999    #BillGates weiß mehr!\n\nhttps://t.co/Bw5s5JJq...
Name: Tweet, Length: 500000, dtype: object

In [6]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [7]:
#load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [8]:
encoded_data = tokenizer.batch_encode_plus(X.values,
                                           add_special_tokens = True,
                                           return_attention_mask = True,
                                           pad_to_max_length = True,
                                           max_length = 280,
                                           return_tensors = 'pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/sebastian/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
encoded_data

{'input_ids': tensor([[  101, 18833,  1011,  ...,     0,     0,     0],
        [  101, 19739,  3240,  ...,     0,     0,     0],
        [  101,  1001,  6207,  ...,     0,     0,     0],
        ...,
        [  101,  1001,  8823,  ...,     0,     0,     0],
        [  101,  2045,  2003,  ...,     0,     0,     0],
        [  101,  1001,  3021,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [10]:
#encode train set
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']

In [11]:
dataset = TensorDataset(input_ids, attention_masks)
len(dataset)

500000

# Load fine-tuned Model

In [12]:
from transformers import BertForSequenceClassification

In [13]:
#load pre-trained BERT
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 4,
                                                      output_attentions = False,
                                                      output_hidden_states = False)
model.load_state_dict(torch.load('model.pt'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

#load train set
dataloader = DataLoader(dataset, batch_size = batch_size)

In [15]:
pred = None

for batch in dataloader:
    batch = tuple(b.to(device) for b in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
        
    outputs = model(**inputs)[0].detach().cpu().numpy()
    if pred is None:
        pred = outputs
    else:
        pred = np.concatenate([pred, outputs])

In [16]:
label_dict = {0:'Positive', 1:'Neutral', 2:'Negative', 3:'Irrelevant'}

In [19]:
df_pred = pd.DataFrame(pred, columns=['Positive', 'Neutral', 'Negative', 'Irrelevant'])
df_pred = pd.concat([df, df_pred], axis=1)
df_pred['Label'] = np.argmax(pred, axis=1)
df_pred['Label'] = df_pred['Label'].replace(label_dict)
df_pred

,Date,Hashtag,Tweet,Postive,Neutral,Negative,Irrelevant,Label
0,2022-11-03 15:03:47+00:00,#Apple,USB-C on an iPhone is probably not gonna happe...,-2.076432,-3.219901,7.715160,-3.449087,Negative
1,2022-11-03 15:03:27+00:00,#Apple,"Guey, que pedo con #Apple no se si sea el únic...",-5.406454,-1.562063,2.487686,5.259759,Irrelevant
2,2022-11-03 15:03:01+00:00,#Apple,#Apple (@Apple) Music offre un petit cadeau au...,-2.524354,8.257416,-1.730636,-3.552223,Neutral
3,2022-11-03 15:02:58+00:00,#Apple,I am using #老虎证券. Check out #Apple (AAPL). Pri...,-2.448110,9.670308,-3.003466,-3.431745,Neutral
4,2022-11-03 15:02:44+00:00,#Apple,Accès coupé à OQEE sur #Apple (@Apple) TV à ca...,-3.639164,-0.332122,7.292725,-4.300461,Negative
...,...,...,...,...,...,...,...,...
499995,2022-03-23 04:11:42+00:00,#billgates,#moderna #fauci #BillGates #Soros #epstein par...,-0.939068,7.612154,-0.252085,-6.350616,Neutral
499996,2022-03-23 02:38:44+00:00,#billgates,@BMi_99999 @GillesWell #BillGates subventionne...,-0.872256,-2.738057,6.752206,-4.453530,Negative
499997,2022-03-23 02:16:04+00:00,#billgates,"#ATENCION 🌐| El globalista, biólogo, virólogo,...",-3.754164,9.147428,-2.854185,-1.573733,Neutral
499998,2022-03-23 01:39:34+00:00,#billgates,There is going to be #exposure of #BillGates. ...,-2.218218,1.219564,5.278355,-4.977199,Negative


In [20]:
df_pred['Label'].value_counts()

Neutral       327671
Irrelevant     65906
Positive       56248
Negative       50175
Name: Label, dtype: int64

In [21]:
df_pred.to_csv('sentiment.csv', index=False)

In [24]:
print(df_pred.loc[499959]['Label'])
df_pred.loc[499959]['Tweet']

Positive


'Honestly when I think of success I see #ElonMusk and #JeffBezos \n\nYou know even the men that came before them #BillGates and #WarrenBuffett\n\nBut am still in a learning curve and fighting my own weakcomings.\n\nI will put in the hours whatever the cost of making my first million.'